##  Training an OCR using CNN + RNN + CTC on Synthetic Images ##
- Now we change the network architecture slightly ; we add a convolutional stack before the BLSTM layer
    - Now your input to the network is not the raw pixel values, But we do steps of convolution and maxpooling and the resultant output is reshpaed to form a Time x featDim structured before it is fed to the network. 
    - The convoultional stack can be increased in depth to get better feature represenations

<b> Upto the model definition part, the code is exactly same as Q3-A . You  may skip reading it but dont forget to run all of them</b>


In [4]:
# =============================================================================
# Use a BRNN + CTC to recognize given word image 
# Network is trained on images rendered using PIL 
# ============================================================================
# 


from __future__ import print_function
from PIL import Image, ImageFont, ImageDraw, ImageEnhance
import numpy as np
import time,math
from time import sleep
import random
import sys,codecs,glob 
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from warpctc_pytorch import CTCLoss
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
random.seed(0)
# TODO - MAKE SURE CTC IS INSTALLED IN ALL MACHINES
use_cuda = torch.cuda.is_available()

if use_cuda:
    print ('CUDA is available')

#use_cuda=False   #uncomment this if you dont want to use cuda variables

CUDA is available


#### vocabulary and the fonts ####
-  loading the lexicon of 90k words
- get the fontslist to be used


In [5]:
#all word images are resized to a height of 32 pixels
imHeight=32 
"""
image width is also set a fixed size
YES. Though RNNS can handle variable length sequences we resize them to fixed width
This is for the ease of batch learning
And it doesnt seem to affect the performance much atleast in our case

Pytorch provides a packed array API incase you want to have variable length sequences within a batch
see the discussion here
https://discuss.pytorch.org/t/simple-working-example-how-to-use-packing-for-variable-length-sequence-inputs-for-rnn/2120/8

"""
#imWidth=100
imWidth=100
fontsList=glob.glob('../../../data/lab2/googleFonts/'+'*.ttf')
vocabFile=codecs.open('../../../data/lab2/lexicon.txt','r')
words = vocabFile.read().split()
vocabSize=len(words)
fontSizeOptions={'16','20','24','28','30','32','36','38'}
batchSize=5
alphabet='0123456789abcdefghijklmnopqrstuvwxyz-'
#alphabet="(3)-"
dict={}
for i, char in enumerate(alphabet):
	dict[char] = i + 1


    


In [6]:
## a simple helper function to compute time since some 'start time'
def time_since(since):
	s = time.time() - since
	m = math.floor(s / 60)
	s -= m * 60
	return '%dm %ds' % (m, s)
 

In [7]:
# return the class labels for each character in the targetsequence 
def Str2Labels(text):
	global dict
	text = [dict[char.lower()] for char in text]
	#print (text)
	length=len(text)
	return text, length
#StrtoLabels("0-1")

### from the predicted sequence of labels for an image, decode the string
# function returns the rawstring and also the decoded string after removing blanks and duplicates

#eg: if labelsequnce you get after an argmax on the output activation matris is  [12,12,0,0,15,0,15,15,0,0]
#then your raw label string would be "bb~~e~ee~~" and the outputstring "bee"
def Labels2Str(predictedLabelSequences):
    bz=predictedLabelSequences.size(0)
    predictedRawStrings=[]
    predictedStrings=[]
    for i in range(0,bz):
        predictedRawString=""
        predictedString=""
        predictedLabelSeq=predictedLabelSequences.data[i,:]
        prevId=1000 #just a large value which is not in the index 
        character=""
        character_raw=""
        for j in range (0, predictedLabelSeq.size(0)):
            idx=predictedLabelSeq[j]
            if (prevId != 1000 or prevId!=idx) :
                if prevId!=idx:
                    if idx==0:
                        character_raw="~"
                        character=""
                    else:
                        character_raw=alphabet[idx-1]
                        character=alphabet[idx-1]
                else:
                    character_raw="~"
                    character=""
                prevId=idx
            else:
                character=""
                if idx==0:
                    character_raw="~"
                else:
                    character_raw=alphabet[idx-1]
                    
                    

            
            predictedString+=character
            predictedRawString+=character_raw
        predictedRawStrings.append(predictedRawString)
        predictedStrings.append(predictedString)
        
    return predictedRawStrings, predictedStrings



def image2tensor(im):
    #returns the pixel values of a PIL image (in 0-1 range) as a numpy 2D array

    (width, height) = im.size
    greyscale_map = list(im.getdata())
    greyscale_map = np.array(greyscale_map, dtype = np.uint8)
    greyscale_map=greyscale_map.astype(float)
    greyscale_map = torch.from_numpy(greyscale_map.reshape((height, width))).float()/255.0
    return greyscale_map


### Render the images, prepare a training batch ###
- renders a batch of word images, from the list of words supplied
- if singleFont is true then only one font would be used to render images. This is useful in case where you want to test overfitting the network to easy examples
- Along with the rendered images, the target strings are converted to corresponding sequence of labels; for example the word "bee" would be converted to [12,15,15] 

In [8]:
def GetBatch ( batchOfWords ):
	"""
	Renders a batch of word images and returns the images along with the corresponding GTs
	Uses PIL to render word images
	font is randomly picked from a set of freely available google fonts
	word is picked from a vocabulary of English words

	"""
	wordImages=[]
	labelSequences=[]
	labelSeqLengths=[]

	for  i,text in enumerate (batchOfWords):
		wordText=text
		#print('text is', text)
		fontName=fontsList[0]
		fontSize='26'
		#fontSize=fontSizeOptions[0]
		fontName=random.sample(fontsList,1)[0]
		fontSize=random.sample(fontSizeOptions,1)[0]
		imageFont = ImageFont.truetype(fontName,int(fontSize))
		textSize=imageFont.getsize(wordText)
		img=Image.new("L", textSize,(255))
		draw = ImageDraw.Draw(img)
		draw.text((0, 0),wordText,(0),font=imageFont)
		img=img.resize((imWidth,imHeight), Image.ANTIALIAS)
		#img.save(text+'.jpeg')

		imgTensor=image2tensor(img)
		imgTensor=imgTensor.unsqueeze(0) # at 0 a new dimenion is added

		wordImages.append(imgTensor)

		labelSeq,l=Str2Labels(wordText)
		labelSequences+=labelSeq
		labelSeqLengths.append(l)
	batchImageTensor=torch.cat(wordImages,0) #now all the image tensors are combined ( we  did the unsqueeze eariler for this cat)	
	#batchImageTensor=torch.transpose(batchImageTensor,1,2)
	labelSequencesTensor=torch.IntTensor(labelSequences)
	labelSeqLengthsTensor=torch.IntTensor(labelSeqLengths)
	return batchImageTensor, labelSequencesTensor, labelSeqLengthsTensor


### Adding a convolutional stack to the BLSTM + CTC Architecure ###
Remember that earlier we were feeding raw pixel columns as inputs at each time step <br>
Here we will use a covolutional stack to get some useful represenations from the word image <br>
And then sequences of this convolutional features are fed to the BLSTM layer above <br>

![CRNN Architecture](crnnstack.png)


In [9]:
# minesh TODO split blstm into a separate class ?

class crnnocr (nn.Module):
    def __init__(self, inputDim, hiddenDim, outputDim,  numLayers, numDirections):
        super(crnnocr, self).__init__()
        self.inputDim=inputDim
        self.hiddenDim=hiddenDim
        self.outputDim=outputDim
        self.numLayers=numLayers
        self.numDirections=numDirections
        # bidirectional= true to make the rnn bidirectional
        #cnn stack
        self.conv1 = nn.Conv2d(1, 64, 3)
        self.conv2 = nn.Conv2d(64, 64, 3)
        
        
        # rnn part
        if numDirections==2:
            self.blstm1=nn.LSTM(384, hiddenDim,numLayers, bidirectional=True, batch_first=True) # first blstm layer takes the image features as inputs
        else:
            self.blstm1=nn.LSTM(384, hiddenDim,numLayers, bidirectional=False, batch_first=True)
        self.linearLayer2=nn.Linear(hiddenDim*numDirections, outputDim) # linear layer at the output
        self.softmax = nn.Softmax()
                
    def forward(self, x ):
        # x will be in shape B x D x T
        x=x.unsqueeze(1) # # we add an extra dimension at 1 for #channels
        #see the input dimension required for conv2s
        #print(x.size())
        
        #print('size of x in the beginning =', x.size()) # batxhSizexnumChannels=1xHxW
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2)) # batchSizex64xH/2-1(W/2-1)x
        #print('size of x after conv1 and pooling =', x.size())
        x = F.max_pool2d(F.relu(self.conv2(x)), 2) # batchSizex64xH/2/2-1x(W/2-1)/2-1
        #print('size of x after conv2 and pooling =', x.size())
        #if input is 50x1x32x100 then it would become 50x64x16*49 and then 50x64x6x23
        #print(x.size())
        x=x.contiguous()
        B,C,D,T=x.size(0), x.size(1), x.size(2), x.size(3)
        #x=x.transpose(2,3) #swapping last two dimensions
        x=x.contiguous()
        x=x.view(B,x.size(1)*x.size(2),-1) # BxC*DXT
        x=x.transpose(1,2) #making T the second dimension
        #print(x.size())
        
        
        lstmOut1, _  =self.blstm1(x ) #x has three dimensions batchSize* seqLen * FeatDim
        B,T,D  = lstmOut1.size(0), lstmOut1.size(1), lstmOut1.size(2)
        lstmOut1=lstmOut1.contiguous()

                
        # output of RNN is reshaped to B*T x D before it is fed to the linear layer
        outputLayerActivations=self.linearLayer2(lstmOut1.view(B*T,D))
        outputSoftMax=self.softmax(outputLayerActivations)
        # the activations are reshaped to B x T x outputDim size
        #then a transpose of B and T since CTC expects the T to be first
        outputLayerActivations= outputLayerActivations.view(B,T,-1).transpose(0,1)
        #if use_cuda:
        #    outputLayerActivations=outputLayerActivations.cuda()
        return outputLayerActivations

### Exercise 1  ###
1. During the above convolution and max pooling operations, what is the stride size used ?
2. Why is first argument to nn.LSTM() is 384 ? What was this value earlier when we didnt have a convolutional stack?
3. Would the value 384 change if If i change the WIdth of the input image ? Give reasons for your answer

In [10]:
###########
# Prepare the synthetic validation data
##############

valWords=['9446567542','hyderabad','golconda','charminar','gachibowli']
valImages, valLabelSeqs, valLabelSeqlens=GetBatch(valWords)
valImages=autograd.Variable(valImages)
valImages=valImages.contiguous()
if use_cuda:
    valImages=valImages.cuda()
valLabelSeqs=autograd.Variable(valLabelSeqs)
#print(valLabelSeqs.data)
valLabelSeqlens=autograd.Variable(valLabelSeqlens)
    

### To handle out of memory error ###
 - First try making the batchSize smaller
 - then you can make the network unidirectional one by setting numDirections=1
 - or change the the number of hidden units
 - in the worst case make imWidth smaller ( this is set at the beginning of the code)
 - if nothing above works uncomment the line use_cuda=False at the beggining 
 - If you are unable to do the training even after trying out above, you should load a pretrained model and see the results (at the end of this notebook)

In [11]:
###########################################
# TRAINING
##################################################
"""
a batch of words are sequentially fetched from the vocabulary
one epoch runs until all the words in the vocabulary are seen once
then the word list is shuffled and above process is repeated
"""
batchSize=5
nHidden=80
nClasses= len(alphabet)
criterion = CTCLoss()

numLayers=2 # the 2 BLSTM layers defined seprately without using numLayers option for nn.LSTM
numDirections=2 # 2 since we need to use a bidirectional LSTM
model = crnnocr(imHeight,nHidden,nClasses,numLayers,numDirections)

optimizer=optim.Adam(model.parameters(), lr=0.001)
start = time.time()
if use_cuda:
    model=model.cuda()
    criterion=criterion.cuda()


for iter in range (0,200):
    avgTrainCost=0
    random.shuffle(words)

    for i in range (0,vocabSize-batchSize+1,batchSize):
    
        model.zero_grad()
        
        batchOfWords=words[i:i+batchSize]
        images,labelSeqs,labelSeqlens =GetBatch(batchOfWords)
        images=autograd.Variable(images)
        #images=autograd.Variable(images)
        images=images.contiguous()
        if use_cuda:
            images=images.cuda()
        labelSeqs=autograd.Variable(labelSeqs)

        labelSeqlens=autograd.Variable(labelSeqlens)
        outputs=model(images)
        outputs=outputs.contiguous()
        outputsSize=autograd.Variable(torch.IntTensor([outputs.size(0)] * batchSize))
        trainCost = criterion(outputs, labelSeqs, outputsSize, labelSeqlens) / batchSize

        avgTrainCost+=trainCost
        if i%500==0:
            avgTrainCost=avgTrainCost/(5000/batchSize)
            #print ('avgTraincost for last 5000 samples is',avgTrainCost)
            avgTrainCost=0
            valOutputs=model(valImages)
#print (valOutputs.size()) 100 X nvalsamoles x 37
            valOutputs=valOutputs.contiguous()
            valOutputsSize=autograd.Variable(torch.IntTensor([valOutputs.size(0)] * len(valWords)))
            valCost=criterion(valOutputs, valLabelSeqs, valOutputsSize, valLabelSeqlens) / len(valWords)
            print ('validaton Cost is',valCost.data[0])


            ### get the actual predictions and compute word error ################
            valOutputs=valOutputs.transpose(0,1)
            # second output of max() is the argmax along the requuired dimension
            _, argMaxActivations= valOutputs.max(2)
            #the below tensor each raw is the sequences of labels predicted for each sample in the batch
            predictedSeqLabels=argMaxActivations.squeeze(2) #batchSize * seqLen
            predictedRawStrings,predictedStrings=Labels2Str(predictedSeqLabels)
            for ii in range(0,5):

                print (predictedRawStrings[ii]+"==>"+predictedStrings[ii])

            #   print (predictedSeqLabels[0,:].transpose(0,0))
            #print(valOutputs_batchFirst[0,0,:])
            print('Time since we began trainiing [%s]' % (time_since(start)))

        optimizer.zero_grad()
        trainCost.backward()
        optimizer.step()
    print( 'completed  iteration no -', iter)
    #if (iter%2==0 or valCost.data[0]  < 6   )  :
    #    iterString=str(iter)
    #    torch.save(model, 'ocrmodel_iter_'+iterString+'.pt')
    #iterString=int(iter)
    #torch.save(model.state_dict(), iterString+'.pth')


validaton Cost is 63.6886291504
j~5~~~~~~~~~~~~~~~~~~~~==>j5
j5~~~~~~~~~~~~~~~~~~~~~==>j5
j5~~~~~~~~~~~~~~~~~~~~~==>j5
j~5~~~~~~~~~~~~~~~~~~~~==>j5
j5~~~~~~~~~~~~~~~~~~~~~==>j5
Time since we began trainiing  0s]
validaton Cost is 39.3794822693
~~~~~~~~~~~~~~~~~~~~~~~==>
~~~~~~~~~~~~~~~~~~~~~~~==>
~~~~~~~~~~~~~~~~~~~~~~~==>
~~~~~~~~~~~~~~~~~~~~~~~==>
~~~~~~~~~~~~~~~~~~~~~~~==>
Time since we began trainiing  1s]
validaton Cost is 40.5147285461
~~~~~~~~~~~~~~~~~~~~~~~==>
~~~~~~~~~~~~~~~~~~~~~~~==>
~~~~~~~~~~~~~~~~~~~~~~~==>
~~~~~~~~~~~~~~~~~~~~~~~==>
~~~~~~~~~~~~~~~~~~~~~~~==>
Time since we began trainiing  3s]
validaton Cost is 41.538898468
~~~~~~~~~~~~~~~~~~~~~~~==>
~~~~~~~~~~~~~~~~~~~~~~~==>
~~~~~~~~~~~~~~~~~~~~~~~==>
~~~~~~~~~~~~~~~~~~~~~~~==>
~~~~~~~~~~~~~~~~~~~~~~~==>
Time since we began trainiing  5s]
validaton Cost is 42.2752532959
~~~~~~~~~~~~~~~~~~~~~~~==>
~~~~~~~~~~~~~~~~~~~~~~~==>
~~~~~~~~~~~~~~~~~~~~~~~==>
~~~~~~~~~~~~~~~~~~~~~~~==>
~~~~~~~~~~~~~~~~~~~~~~~==>
Time since we be

KeyboardInterrupt: 

### Loading a pretrained model and testing the validation data on it ###
In case your networks takes lot of time to converge, we have a pretrained model for you. <br>


In [11]:
#  load a saved model and test our test/validation data on it #
use_cuda=True #since the saved model is a gpu model
model = torch.load("../../../data/lab2/crnnmodel.pt")
criterion = CTCLoss()
if use_cuda:
    model=model.cuda()
    criterion=criterion.cuda()


valWords=['9446567456','hyderabad','golconda','charminar','gachibowli']
valImages, valLabelSeqs, valLabelSeqlens=GetBatch(valWords)
valImages=autograd.Variable(valImages)
valImages=valImages.contiguous()
if use_cuda:
    valImages=valImages.cuda()
valLabelSeqs=autograd.Variable(valLabelSeqs)
#print(valLabelSeqs.data)
valLabelSeqlens=autograd.Variable(valLabelSeqlens)
valOutputs=model(valImages)
valOutputs=valOutputs.contiguous()
valOutputsSize=autograd.Variable(torch.IntTensor([valOutputs.size(0)] * len(valWords)))
valCost=criterion(valOutputs, valLabelSeqs, valOutputsSize, valLabelSeqlens) / len(valWords)
print ('validaton Cost is',valCost.data[0])


# valOutputs is in TxBxoutputDim size we make it BxTxoutputDIm
valOutputs_batchFirst=valOutputs.transpose(0,1)
# second output of max() is the argmax along the requuired dimension
_, argMaxActivations= valOutputs_batchFirst.max(2)
#the below tensor each raw is the sequences of labels predicted for each sample in the batch
predictedSeqLabels=argMaxActivations.squeeze(2) #batchSize * seqLen 
predictedRawStrings,predictedStrings=Labels2Str(predictedSeqLabels)
#print the predicted raw string and the decoded string for the valimages
for ii in range(0,5):

    print (predictedRawStrings[ii]+"==>"+predictedStrings[ii])

validaton Cost is 17.3930625916
s~u~~a~a~~s~s~~i~a~~s~e==>suaassiase
h~~y~d~~e~~r~a~~b~~a~~d==>hyderabad
g~~o~~l~c~~o~~~n~~d~~a~==>golconda
c~h~~a~~r~~n~li~n~~a~~r==>charnlinar
g~~a~c~~h~i~~b~o~~u~l~i==>gachibouli


### Exercise ###
0. Did you notice that length of the predictedRawstrings is shorter in this case? For example rawprediction of hyderabad is "h~~y~d~~e~~r~a~~b~~a~~d". Eariler the raw string was of length=100 (=imWidth). Why is the length shorter now?
1. Modify the Labels2Str() function to return the word accuracy along with the other returned values
2. Have you noticed that numbers are poorly recognized using both our architecures. Can you reason why the recognition accuracies are poorer for numbers
    - hint -look at your vocabulary --  find out how many 'words' with numbers are in there
3. In general when the image is a sequence of numbers, RNNs perform poorly compared to its performance on recognizing words in a language. What could be the reason why RNNs perform poorer on a sequence of numbers
    - hint - Rnn's memory itself is the villain
4. You were introduced to beam search as an an alternative to argMax decoding while solving seq2seq problems. Can you write a beam search decoding for our problem and see if the recognition results are different with beam search (You may use existing code. librararies for beam search)